In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import gc
import warnings
from bayes_opt import BayesianOptimization

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from datetime import datetime
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod(
            (datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [ ]:
train = pd.read_csv("../input/train_df.csv", index_col=0)
target = train['target']


In [ ]:
features = ["new_month_lag_max","hist_month_diff_max","new_month_lag_min","hist_month_diff_min","hist_quarter_max","new_month_max","hist_month_max",
                "hist_month_lag_min","new_category_1_mean","feature_1","new_month_min","new_category_1_sum","new_Black_Friday_2017_mean",
                "month_lag_min","new_quarter_min","hist_month_diff_mean","new_Children_day_2017_mean","quarter","new_month_lag_mean",
                "new_month_mean","feature_min","hist_category_1_sum","new_quarter_max","new_purchase_date_uptonow","hist_category_3_sum",
                "new_purchase_amount_max","hist_purchase_date_diff","new_Christmas_Day_2017_mean","hist_quarter_min",
                "hist_purchase_date_uptonow","new_weekofyear_mean","hist_duration_min","new_purchase_date_diff","hist_category_1_mean","hist_installments_sum",
                "elapsed_time","category_1_mean","days_feature3","new_duration_max","feature_3","hist_fathers_day_2017_mean","purchase_amount_diff_v2",
                "hist_merchant_id_nunique","hist_authorized_flag_mean","hist_last_buy","hist_month_lag_max","hist_duration_max","month_lag_max",
                "hist_Valentine_Day_2017_mean","duration_min","hist_amount_month_ratio_min","card_id_cnt_ratio","hist_duration_mean",
                "hist_CLV","new_merchant_category_id_nunique","hist_month_min","purchase_amount_mlag_ratio","new_purchase_amount_mean","new_card_id_size","hist_purchase_amount_sum",
                "new_day_mean","new_price_min","hist_purchase_date_uptomin","hist_month_lag_mean","hist_Mothers_Day_2017_mean","hist_merchant_category_id_nunique",
                "month_diff_mean","new_duration_mean","hist_subsector_id_nunique","month_lag_mean","hist_category_3_mean_mean","hist_day_nunique",
                "new_quarter_mean","new_purchase_date_average","installments_total","new_amount_month_ratio_max","purchase_amount_ratio_v3",
                "purchase_amount_diff_v3","new_installments_max","new_installments_mean","hist_month_lag_var","hist_card_id_count",
                "hist_purchase_date_average","new_day_min","feature_sum","days_feature2","hist_Children_day_2017_mean","new_duration_min",
                "new_merchant_id_nunique","new_card_id_count","new_last_buy","new_CLV","feature_mean","card_id_total","new_purchase_date_uptomin",
                "new_month_lag_var","hist_price_sum","price_mean","installments_mean","new_category_3_mean_mean"]
len(features)

In [ ]:
def LGB_CV(max_depth,learning_rate,num_leaves,min_data_in_leaf,colsample_bytree,subsample,reg_lambda,reg_alpha):
    folds = KFold(n_splits=5, shuffle=True, random_state=15)
    oof = np.zeros(train.shape[0])
    
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
        print("fold n°{}".format(fold_))
        trn_data = lgb.Dataset(train.iloc[trn_idx][features],
                               label=target.iloc[trn_idx])
        val_data = lgb.Dataset(train.iloc[val_idx][features],
                               label=target.iloc[val_idx])
        
        param = {'task': 'train',
                'boosting': 'goss',
                'objective': 'regression',
                'metric': 'rmse',
                'learning_rate' : learning_rate,
                'subsample': subsample,
                'max_depth': int(max_depth),
                'top_rate': 0.9064148448434349,
                'num_leaves': int(num_leaves),
                'other_rate': 0.0721768246018207,
                'reg_alpha': int(reg_alpha),
                'colsample_bytree': colsample_bytree,
                'min_split_gain': 9.820197773625843,
                'reg_lambda': int(reg_lambda),
                'min_data_in_leaf': int(min_data_in_leaf),
                'verbose': -1,
                'seed':123,
                'bagging_seed':456,
                'min_child_weight': 41.9612869171337,
                'drop_seed':789}
        ,
        clf = lgb.train(param,
                        trn_data,
                        10000,
                        valid_sets = [trn_data, val_data],
                        verbose_eval=500,
                        early_stopping_rounds = 200)
        
        oof[val_idx] = clf.predict(train.iloc[val_idx][features],
                                   num_iteration=clf.best_iteration)
        
        del clf, trn_idx, val_idx
        gc.collect()
        
    return -mean_squared_error(oof, target)**0.5

In [ ]:
LGB_BO = BayesianOptimization(LGB_CV, {
    'learning_rate':(0.01,0.1),
    'max_depth': (4, 10),
    'num_leaves': (5, 130),
    'min_data_in_leaf': (10, 150),
    'colsample_bytree': (0.6, 1.0),
    'subsample': (0.7, 1.0),
    'reg_lambda': (2,8),
    'reg_alpha':(2,8)
    })

In [ ]:
print('-'*126)

start_time = timer(None)
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    LGB_BO.maximize(init_points=2, n_iter=30, acq='ei', xi=0.0)
timer(start_time)

In [ ]:
print('-'*130)
print('Final Results')
print('Maximum  value: %f' % LGB_BO.max['target'])
print('Best  parameters: ', LGB_BO.max['params'])

history_df = pd.DataFrame([x['params'] for x in LGB_BO.res])
history_df2 = pd.DataFrame([x['target'] for x in LGB_BO.res])
history_df = pd.concat((history_df, history_df2), axis=1)
history_df.rename(columns = { 0 : 'gini'}, inplace=True)
history_df.to_csv('ParametersOptimization.csv')